In [ ]:
# ref. https://huggingface.co/nvidia/speakerverification_en_titanet_large
# Now to check if two audio files are from the same speaker or not, simply do:
# speaker_model.verify_speakers("an255-fash-b.wav","cen7-fash-b.wav")
import nemo.collections.asr as nemo_asr

speaker_model = nemo_asr.models.EncDecSpeakerLabelModel.from_pretrained("nvidia/speakerverification_en_titanet_large")


In [ ]:
# verify that two sounds are from the same speaker
#speaker_model.verify_speakers("an255-fash-b.wav","cen7-fash-b.wav")

# json format
#{"audio_filepath": "<absolute path to dataset>/audio_file.wav", "duration": "duration of file in sec", "label": "speaker_id"}
# /home/kozi/Documents/amicorpus/extracted/67ea94407e8e7c9c26af373f726de533eb38a5203aaf9b20e5fab0325364d827/TS3003c/
# https://groups.inf.ed.ac.uk/ami/corpus/participantids.shtml
    # example file name: AMI_EN2002a_H00_MEE073_0023463_0023492.wav
    # participant ID: MEE073, indicating that the speaker is a male, 2 letters for region and 3 numbers to create a unique identifier

# use this manifest
#python <NeMo_root>/examples/speaker_tasks/recognition/extract_speaker_embeddings.py --manifest=manifest.json


In [ ]:
# run this command to run NeMo diarization
#python3 /home/kozi/Documents/NeMo/examples/speaker_tasks/recognition/extract_speaker_embeddings.py --manifest=/home/kozi/.cache/huggingface/datasets/downloads/nemo_ami_manifest_small.json --embedding_dir=/home/kozi/.cache/huggingface/datasets/downloads/embeddings_out


In [ ]:
# view the generated embedding which is an unusable list of floats obviously; this ipynb is a dead end
import pickle

with open('/home/kozi/.cache/huggingface/datasets/downloads/embeddings_out/embeddings/nemo_ami_manifest_small_embeddings.pkl', 'rb') as file:
    loaded_data = pickle.load(file)

print(loaded_data)

In [38]:
# ref. https://huggingface.co/datasets/edinburghcstr/ami/resolve/main/annotations/eval/text

# converts a folder of AMI to nemo's manifest JSON style (only the manifest since audio is already wav)

# unused, everything works with RTTM and this is only creates a simple manifest file

import os                                                                      
import subprocess                                                          
import json                                                                    
                                                                            

def get_wav_duration(file_path):                                               
    cmd = "ffprobe -v error -show_entries format=duration -of default=noprint_wrappers=1:nokey=1".split()     
    cmd.append(file_path)                                                      
    output = subprocess.check_output(cmd)                                      
    duration = float(output)                                                   
    return round(duration, 3)                                                  
                                                                            
def main(directory):        
    transcriptions = {}
    with open(os.path.join(directory, "eval-text.txt"), "r") as eval_text:
        for row in eval_text:
            words = row.split(" ")
            key = words[0].upper()
            transcription = " ".join(words[1:])
            transcriptions[key] = transcription

    with open('/home/kozi/.cache/huggingface/datasets/downloads/transcriptions.json', 'w') as json_file:                                                                  
        json.dump(transcriptions, json_file, indent=4)

    nemo_data = []                                                     
    for item in os.listdir(os.path.join(directory, "extracted")):
        subdir = os.path.join(os.path.join(directory, "extracted"), item)
        if not os.path.isdir(subdir):
            print("not dir {}".format(subdir))
        else:
            for dirpath, dirnames, wav_files in os.walk(subdir):
                for wav_file in wav_files:
                    if(not "eval" in wav_file):
                        #print("skipped {}".format(wav_file))
                        continue
                    else:
                        print("wav_file found {}".format(wav_file))

                    if not (wav_file.endswith('.wav')): # and os.path.isfile(wav_file) and 
                        print("skipped {}".format(wav_file))
                        continue
                    try:
                        # format: filename --> key; eval_ami_en2002d_h01_feo072_0181679_0181767.wav --> AMI_EN2002d_H01_FEO072_0181679_0181767
                        key = "_".join(wav_file.upper().split("_")[1:])[:-4]
                        print(key)
                        transcription = transcriptions[key]
                        abs_path = os.path.abspath(os.path.join(dirpath, wav_file)) 
                        duration = get_wav_duration(abs_path) 
                        label = key.split("_")[3]
                        nemo_entry = {
                            "audio_filepath": abs_path,
                            "duration": duration,                                      
                            "label": label,
                            "text": transcription#.encode('utf-8')  
                        }
                        nemo_data.append(nemo_entry)
                    finally:
                        print("something is missing, wavfile: {}".format(os.path.abspath(wav_file)))

    nemo_manifest = os.path.join(directory, "nemo_manifest.json")
    print("writing file {}".format(nemo_manifest))
    with open(nemo_manifest, "w") as fout:
        for m in nemo_data:
            # ensure_ascii=True is the default and even speech_transcribe will use escaped umlauts, but this way the manifest is human-readable
            fout.write(json.dumps(m, ensure_ascii=False) + "\n")

main("/home/kozi/.cache/huggingface/datasets/downloads/")


wav_file found eval_ami_en2002d_h01_feo072_0181679_0181767.wav
AMI_EN2002D_H01_FEO072_0181679_0181767
something is missing, wavfile: /home/kozi/Documents/_onlab_git/ami/eval_ami_en2002d_h01_feo072_0181679_0181767.wav
wav_file found eval_ami_en2002d_h03_mee073_0068407_0068548.wav
AMI_EN2002D_H03_MEE073_0068407_0068548
something is missing, wavfile: /home/kozi/Documents/_onlab_git/ami/eval_ami_en2002d_h03_mee073_0068407_0068548.wav
wav_file found eval_ami_en2002d_h00_feo070_0042040_0042072.wav
AMI_EN2002D_H00_FEO070_0042040_0042072
something is missing, wavfile: /home/kozi/Documents/_onlab_git/ami/eval_ami_en2002d_h00_feo070_0042040_0042072.wav
wav_file found eval_ami_en2002d_h01_feo072_0090892_0090996.wav
AMI_EN2002D_H01_FEO072_0090892_0090996
something is missing, wavfile: /home/kozi/Documents/_onlab_git/ami/eval_ami_en2002d_h01_feo072_0090892_0090996.wav
wav_file found eval_ami_en2002d_h03_mee073_0029991_0030027.wav
AMI_EN2002D_H03_MEE073_0029991_0030027
something is missing, wavfile: